# backtesting Scratch

Notebook for experimenting with the `backtesting` package

## Libraries

In [1]:
# stdlib

In [1]:
# 3rd party
import pandas as pd
import ta


from backtesting import Strategy, Backtest
from backtesting.lib import resample_apply
from backtesting.lib import crossover

In [2]:
# project

## Strategy Setup

In [9]:
SMA = lambda x, n: ta.trend.sma_indicator(close=pd.Series(x), window=n)
RSI = lambda x, n: ta.momentum.rsi(close=pd.Series(x), window=n)

In [10]:
class System(Strategy):
    d_rsi = 30  # Daily RSI lookback periods
    w_rsi = 30  # Weekly
    level = 70
    
    def init(self):
        # Compute moving averages the strategy demands
        self.ma10 = self.I(SMA, self.data.Close, 10)
        self.ma20 = self.I(SMA, self.data.Close, 20)
        self.ma50 = self.I(SMA, self.data.Close, 50)
        self.ma100 = self.I(SMA, self.data.Close, 100)
        
        # Compute daily RSI(30)
        self.daily_rsi = self.I(RSI, self.data.Close, self.d_rsi)
        
        # To construct weekly RSI, we can use `resample_apply()`
        # helper function from the library
        self.weekly_rsi = resample_apply(
            'W-FRI',
            RSI,
            self.data.Close,
            self.w_rsi
        )
        
        
    def next(self):
        price = self.data.Close[-1]
        
        # If we don't already have a position, and
        # if all conditions are satisfied, enter long.
        if (not self.position and
            self.daily_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.daily_rsi[-1] and
            self.ma10[-1] > self.ma20[-1] > self.ma50[-1] > self.ma100[-1] and
            price > self.ma10[-1]):
            
            # Buy at market price on next open, but do
            # set 8% fixed stop loss.
            self.buy(sl=.92 * price)
        
        # If the price closes 2% or more below 10-day MA
        # close the position, if any.
        elif price < .98 * self.ma10[-1]:
            self.position.close()

In [11]:
from backtesting.test import GOOG

backtest = Backtest(GOOG, System, commission=.002)
backtest.run()
backtest.plot(filename='outputs/test.html')

Row(id='1626', ...)

In [12]:
backtest.optimize(
    d_rsi=range(10, 35, 5),
    w_rsi=range(10, 35, 5),
    level=range(30, 80, 10)
)

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   27.094972
Equity Final [$]                  23304.82018
Equity Peak [$]                   24136.96578
Return [%]                         133.048202
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                   10.435363
Volatility (Ann.) [%]               14.731717
Sharpe Ratio                          0.70836
Sortino Ratio                        1.186656
Calmar Ratio                         0.606959
Max. Drawdown [%]                  -17.192867
Avg. Drawdown [%]                   -3.596989
Max. Drawdown Duration      693 days 00:00:00
Avg. Drawdown Duration       71 days 00:00:00
# Trades                                   30
Win Rate [%]                             60.0
Best Trade [%]                      25.034669
Worst Trade [%]                     -7.914445
Avg. Trade [%]                    

In [13]:
backtest.plot()

Row(id='3099', ...)